In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bot-iot/data_26.csv
/kaggle/input/bot-iot/data_33.csv
/kaggle/input/bot-iot/data_49.csv
/kaggle/input/bot-iot/data_44.csv
/kaggle/input/bot-iot/data_38.csv
/kaggle/input/bot-iot/data_22.csv
/kaggle/input/bot-iot/data_30.csv
/kaggle/input/bot-iot/data_57.csv
/kaggle/input/bot-iot/data_46.csv
/kaggle/input/bot-iot/data_58.csv
/kaggle/input/bot-iot/data_21.csv
/kaggle/input/bot-iot/data_59.csv
/kaggle/input/bot-iot/data_32.csv
/kaggle/input/bot-iot/data_69.csv
/kaggle/input/bot-iot/data_19.csv
/kaggle/input/bot-iot/data_47.csv
/kaggle/input/bot-iot/data_14.csv
/kaggle/input/bot-iot/data_9.csv
/kaggle/input/bot-iot/data_52.csv
/kaggle/input/bot-iot/data_18.csv
/kaggle/input/bot-iot/data_53.csv
/kaggle/input/bot-iot/data_73.csv
/kaggle/input/bot-iot/data_63.csv
/kaggle/input/bot-iot/data_70.csv
/kaggle/input/bot-iot/data_17.csv
/kaggle/input/bot-iot/data_7.csv
/kaggle/input/bot-iot/data_67.csv
/kaggle/input/bot-iot/data_names.csv
/kaggle/input/bot-iot/data_11.csv
/kaggle/input

# check the files exist.. also see the sample format for reference..

In [70]:
import pandas as pd
sample_file = "/kaggle/input/bot-iot/data_11.csv"
sample_df = pd.read_csv(sample_file)
print(sample_df.memory_usage().sum() / (1024**3), "GB")  # Size in GB
print(sample_df.shape)  # Rows, columns

0.26077044010162354 GB
(1000000, 35)


# Using the dask based dataframe and parquent file for ensuring the size efficiency of the file and to be able to handle the low computation and ram of a avgerage system..  **Apache Parquet is an open source, column-oriented data file format designed for efficient data storage and retrieval.**  total dataset size is ~15 gb converted into ~1Gb(70%) of the dataset   

# dask approach

In [71]:
import dask.dataframe as dd
import pandas as pd

# Specify only the columns we need to reduce memory footprint
selected_columns = [
    'pkts', 'bytes', 'sbytes', 'dbytes', 'dur', 'rate', 'mean', 'stddev', 'sum', 
    'proto', 'state', 'stime', 'category'  # Ensure 'category' is included
]

# Load all CSV files with Dask
df = dd.read_csv("/kaggle/input/bot-iot/*.csv", usecols=selected_columns, dtype={'category': 'object'})

# Basic preprocessing (lazy evaluation)
df = df.dropna()  # Drops NaN rows when computed

# Encode categorical features (excluding 'category' to retain original class labels)
df['proto'] = df['proto'].map_partitions(lambda x: pd.Categorical(x).codes)
df['state'] = df['state'].map_partitions(lambda x: pd.Categorical(x).codes)

# Compute class distribution before encoding 'category'
class_counts = df['category'].value_counts().compute()

# Print class counts
print("Class Distribution:\n", class_counts)

# Encode 'category' after checking distributions
df['category'] = df['category'].map_partitions(lambda x: pd.Categorical(x, categories=class_counts.index).codes)

# Print preview of processed dataframe
print(df.head())


KeyboardInterrupt: 

In [ ]:
import dask.dataframe as dd

# Define the path to CSV files
# csv_path = "/kaggle/input/bot-iot/*.csv"

# Read only the 'category' column, ensuring it's treated as a string
# df = dd.read_csv(csv_path, usecols=['category'], dtype={'category': 'object'})

# Compute category distribution


# Display class counts
print(df['category'].value_counts().compute())


# type casting to lower datatypes to make use of less memory(batch efficiency)

In [ ]:

df = df.astype({
    'pkts': 'int16', 'bytes': 'int32', 'sbytes': 'int32', 'dbytes': 'int32',
    'dur': 'float32', 'rate': 'float32', 'mean': 'float32', 'stddev': 'float32', 'sum': 'float32',
    'proto': 'int8', 'state': 'int8', 'stime': 'float32', 'category': 'int8'
})

# down sizing the dataset using 70% of the dataset to ensure the organility

In [ ]:
# For Dask
df = df.sample(frac=0.7, random_state=42)

In [ ]:
df.to_parquet("/kaggle/working/processed_data.parquet")  # Dask
df = dd.read_parquet("/kaggle/working/processed_data.parquet")  # Reload later

# Dataset Info and checking for null values

In [ ]:
import dask.dataframe as dd

# Load the Parquet file (if not already loaded)
df = dd.read_parquet("/kaggle/working/processed_data.parquet")

# Get basic information about the dataset
print(df.info())  # Check columns and data types
print(df.memory_usage(deep=True).compute())  # Check memory usage
print(df.head())  # Preview first few rows

# Check for missing values
missing_values = df.isnull().sum().compute()
print("Missing Values:\n", missing_values)


# class counts

In [ ]:
# Compute class distribution
class_counts = df['category'].value_counts().compute()
print("Class Distribution:\n", class_counts)


# numeric cols Distributions

In [ ]:
import matplotlib.pyplot as plt

# Select a few numeric columns to visualize
numeric_columns = ['pkts', 'bytes', 'sbytes', 'dbytes', 'dur', 'rate']

# Convert Dask DF to Pandas (for small samples)
sample_df = df[numeric_columns].sample(frac=0.001, random_state=42).compute()

# Plot histograms
sample_df.hist(figsize=(12, 8), bins=50)
plt.suptitle("Feature Distributions")
plt.show()


# Detect and Handle Skewed Features

In [ ]:
from scipy.stats import skew

# Check skewness of numeric features
skewness = sample_df.skew()
print("Feature Skewness:\n", skewness)

# Apply log transformation if necessary
for col in skewness[abs(skewness) > 1].index:
    sample_df[col] = sample_df[col].apply(lambda x: np.log1p(x))  # Log transformation

# Plot transformed distributions
sample_df.hist(figsize=(12, 8), bins=50)
plt.suptitle("Log-Transformed Feature Distributions")
plt.show()


# Class Distribution (Bar Chart)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute class distribution
class_counts = df['category'].value_counts().compute()

# Plot class distribution
plt.figure(figsize=(10, 5))
sns.barplot(x=class_counts.index, y=class_counts.values, palette="viridis")
plt.xlabel("Attack Categories")
plt.ylabel("Frequency")
plt.title("Class Distribution in Bot-IoT Dataset")
plt.xticks(rotation=45)
plt.show()


# Feature Distributions (Histograms)

In [ ]:
# Select numeric columns
numeric_columns = ['pkts', 'bytes', 'sbytes', 'dbytes', 'dur', 'rate']

# Compute a small sample for visualization
sample_df = df[numeric_columns].sample(frac=0.001, random_state=42).compute()

# Plot histograms
sample_df.hist(figsize=(12, 8), bins=50, edgecolor='black', alpha=0.7)
plt.suptitle("Feature Distributions in Bot-IoT Dataset")
plt.show()


# Boxplots to Identify Outliers

In [ ]:
# Plot boxplots for numeric features
plt.figure(figsize=(12, 6))
sample_df.boxplot(column=numeric_columns, vert=False)
plt.title("Feature Boxplots - Detecting Outliers")
plt.show()


# Pairplot for Feature Relationships

In [ ]:
import seaborn as sns

# Take a small sample for visualization
sample_df = sample_df.sample(1000, random_state=42)  # Reduce size for visualization

# Pairplot
sns.pairplot(sample_df, diag_kind='kde')
plt.suptitle("Pairplot of Selected Features", y=1.02)
plt.show()


# Distribution of Packet Durations Across Attack Types


In [ ]:
# Convert to Pandas before plotting
df_sample = df[['category', 'dur']].sample(frac=0.001, random_state=42).compute()

# Now, plot using Seaborn
plt.figure(figsize=(12, 6))
sns.boxplot(x=df_sample['category'], y=df_sample['dur'], palette="coolwarm")
plt.xlabel("Attack Category")
plt.ylabel("Packet Duration")
plt.title("Distribution of Packet Durations Across Attack Types")
plt.xticks(rotation=45)
plt.show()


# Protocol Usage in Different Attacks

In [ ]:
# Sample data for better visualization
sample_df = df.sample(frac=0.001, random_state=42).compute()

# Countplot for protocol usage
plt.figure(figsize=(12, 6))
sns.countplot(x='proto', hue='category', data=sample_df, palette="coolwarm")
plt.xlabel("Protocol")
plt.ylabel("Count")
plt.title("Protocol Usage in Different Attack Categories")
plt.xticks(rotation=45)
plt.legend(title="Attack Type", bbox_to_anchor=(1, 1))
plt.show()


# Check the class distribution

In [ ]:
import dask.dataframe as dd

# Load the Parquet file using Dask
df = dd.read_parquet("/kaggle/working/processed_data.parquet")

# Compute class distribution
class_counts = df['category'].value_counts().compute()

# Print class counts
print(class_counts)


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Compute Dask DataFrame to pandas (after filtering/sampling)
df = df.compute()  # Ensure this fits in ~15 GB post-optimization

# Split features and target
# X = df.drop(columns=['category']).values
# y = df['category'].values

# # Convert to tensors
# X_tensor = torch.FloatTensor(X)
# y_tensor = torch.LongTensor(y)

# # Create DataLoader
# dataset = TensorDataset(X_tensor, y_tensor)
# train_loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)  # Use 2 workers for Kaggle CPU

# Check the memory usage make shure we dont crash on ram

In [ ]:
import psutil
print(f"Memory used: {psutil.virtual_memory().percent}%")

# Feature extraction using tree based classifer and l1 regularization (since  dealing with high-dimensional datasets, noisy data, and situations where computational efficiency is essential.)

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import Lasso
import seaborn as sns
import matplotlib.pyplot as plt

# Path to folder containing multiple CSV files
data_folder = '/kaggle/input/bot-iot'  # Update with your folder path

# Read a sample from all CSVs to analyze feature importance
sample_size = 500000  # Adjust based on memory
all_data = []

def sample_data(file_path, sample_size):
    chunk = pd.read_csv(file_path, nrows=sample_size, low_memory=False)
    return chunk

for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_folder, filename)
        print(f"Sampling from {filename}...")
        df_sample = sample_data(file_path, sample_size // len(os.listdir(data_folder)))
        all_data.append(df_sample)

# Merge sampled data
df = pd.concat(all_data, ignore_index=True)

# Remove unnecessary columns
null_cols = ['dco', 'sco', 'doui', 'soui', 'dmac', 'smac']  # Replace with actual null columns
df.drop(columns=['pkSeqID'] + null_cols, inplace=True, errors='ignore')

# Remove rows with missing values
df.dropna(inplace=True)

# Detect categorical columns dynamically
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Convert categorical columns to strings before encoding
for col in categorical_cols:
    df[col] = df[col].astype(str)

# Encode categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Define target variable
X = df.drop(columns=['category'])  # Update 'Label' to actual target column
y = df['category']

# Correlation Analysis
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), cmap='coolwarm', annot=False)
plt.title("Feature Correlation Matrix")
plt.show()

# Mutual Information
mi_scores = mutual_info_classif(X, y, discrete_features=False)
mi_scores = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(10, 6))
mi_scores.plot(kind='bar')
plt.title("Mutual Information Scores")
plt.show()

# Random Forest Feature Importance
rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
rf.fit(X, y)
feature_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(10, 6))
feature_importances.plot(kind='bar')
plt.title("Random Forest Feature Importance")
plt.show()

# Lasso Regression for Feature Selection
lasso = Lasso(alpha=0.01)
lasso.fit(X, y)
lasso_importance = pd.Series(np.abs(lasso.coef_), index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(10, 6))
lasso_importance.plot(kind='bar')
plt.title("Lasso Regression Feature Importance")
plt.show()

# Select the Top 10 Most Important Features
# top_features = feature_importances.head(10).index.tolist()
# print("Top 10 Selected Features:", top_features)


In [ ]:
# Get top 10 features from both methods
rf_top_features = set(feature_importances.head(10).index.tolist())
lasso_top_features = set(lasso_importance.head(10).index.tolist())

print("Top 10 Selected Features (Intersection of RF and Lasso):", rf_top_features)
print("Top 10 Selected Features (Intersection of RF and Lasso):", lasso_top_features)

# Find the intersection
top_features = rf_top_features.intersection(lasso_top_features)

print("Top 10 Selected Features (Intersection of RF and Lasso):", top_features)


# the below feature that are selected are the combination from feature selection process and Domain knowledge ..

In [ ]:
df = dd.read_parquet("/kaggle/working/processed_data.parquet")  

# current data preprocessing appraoch.. 
#** intial dataset distributions
 Class 0: ~450,000 samples
Class 1: 22 samples
Class 2: ~525,000 samples
Class 3: ~124 samples
Class 4: ~25,000 samples

then test set is defined as follewed:
thing is 

Using the proportions from the test set:

Class 0: 
(449,939/1,000,000)×4,000,000 ≈ 1,799,756

Class 1: 
(22/1,000,000)×4,000,000 ≈ 88

Class 2: 
(525,176/1,000,000) ×4,000,000 ≈  2,100,704

Class 3: 
(124/1,000,000)×4,000,000 ≈ 496

Class 4: 
(24,739/1,000,000) ×4,000,000 ≈98,956

SO training set  before sampling were
 Class 0: 1,799,756 (~45.0%)
 Class 1: 88 (~0.002%)
 Class 2: 2,100,704 (~52.5%)
 Class 3: 496 (~0.012%)
 Class 4: 98,956 (~2.5%)

 step2: under sampling the majority class
  RandomUnderSampler to undersample the majority classes (Classes 0, 2, and 4) to 50,000 samples each
  Class Distribution After Undersampling:

Class 0: 50,000 (~49.2%)
Class 1: 88 (~0.09%)
Class 2: 50,000 (~49.2%)
Class 3: 496 (~0.49%)
Class 4: 50,000 (~49.2%)
Total samples: 50,000 + 88 + 50,000 + 496 + 50,000 = 150,584


step3: oversampling with the adasyn due to the (reason:Unlike SMOTE, which generates synthetic samples uniformly across the feature space, ADASYN focuses on generating more synthetic samples for minority instances that are harder to classify correctly, based on their level of difficulty in classification)

Final Class Distribution in Training Set (After ADASYN):

Class 0: 50,000 (~12.5%)
Class 1: 75,000 (~18.75%)
Class 2: 50,000 (~12.5%)
Class 3: 125,000 (~31.25%)
Class 4: 75,000 (~18.75%)
Total samples: 50,000 + 75,000 + 50,000 + 125,000 + 75,000 = 375,000

# data preprocessing 

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
import gc

# Step 1: Inspect the 'category' column
print("Unique values in 'category':", np.unique(df['category']))
print("Class distribution before sampling:\n", df['category'].value_counts())

# Step 2: Combine with Chebyshev distance
X_static = np.array(df[['pkts', 'bytes', 'sbytes', 'dbytes', 'rate', 'mean', 'stddev', 'sum', 'proto', 'state']].values, dtype=np.float32)
X_temporal = np.array(df[['stime', 'dur']].values, dtype=np.float32)
X_static_mean = X_static.mean(axis=1)
X_static_std = X_static.std(axis=1)
X_static_reduced = np.vstack([X_static_mean, X_static_std]).T
chebyshev_dist = np.max(np.abs(X_static_reduced - X_temporal), axis=1)
X_combined = np.hstack((X_static, chebyshev_dist.reshape(-1, 1)))
y = np.array(df['category'].values, dtype=np.int64)

# Free memory
del df, X_static, X_temporal, X_static_mean, X_static_std, X_static_reduced, chebyshev_dist
gc.collect()

# Step 3: Sample to reduce size (target ~5M samples total)
np.random.seed(42)
sample_indices = np.random.choice(len(X_combined), size=5_000_000, replace=False)
X_combined = X_combined[sample_indices]
y = y[sample_indices]

# Compute class distribution after sampling
class_counts_sampled = np.bincount(y)
print("Class distribution after sampling:", {i: count for i, count in enumerate(class_counts_sampled)})

# Step 4: Train-test split BEFORE sampling
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=42, stratify=y
)

# Free memory
del X_combined, y
gc.collect()

# Compute class distribution in training set
class_counts_train = np.bincount(y_train)
print("Class distribution in training set:", {i: count for i, count in enumerate(class_counts_train)})

# Step 5: Hybrid Sampling to Balance the Training Set
# 5.1: Undersample the majority classes (Classes 0, 2, and 4)
undersampler = RandomUnderSampler(
    sampling_strategy={
        0: 50_000,  # Target 50K for Class 0
        2: 50_000,  # Target 50K for Class 2
        4: 50_000   # Target 50K for Class 4
    },
    random_state=42
)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

# Free memory
del X_train, y_train
gc.collect()

# Compute class distribution after undersampling
class_counts_under = np.bincount(y_train_under)
print("Class distribution after undersampling:", {i: count for i, count in enumerate(class_counts_under)})

# 5.2: Apply ADASYN to oversample the minority classes (Classes 1, 3)
adasyn = ADASYN(
    random_state=42,
    sampling_strategy={
        0: 50_000,  # Already at 50K
        1: 75_000,  # Oversample to 75K
        2: 50_000,  # Already at 50K
        3: 100_000, # Further reduced oversampling for Class 3
        4: 75_000   # Already at 75K
    }
)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_under, y_train_under)

# Add random noise to synthetic samples to increase diversity
noise = np.random.normal(0, 0.02, X_train_resampled.shape)
X_train_resampled += noise

# Free memory
del X_train_under, y_train_under
gc.collect()

# Compute class distribution after ADASYN
class_counts_resampled = np.bincount(y_train_resampled)
print("Class distribution after ADASYN:", {i: count for i, count in enumerate(class_counts_resampled)})


# Step 6: Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# Free memory
del X_train_resampled, X_test
gc.collect()

# Step 7: Convert to tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_resampled, dtype=torch.int64)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)

# Free memory
del X_train_scaled, X_test_scaled, y_train_resampled, y_test
gc.collect()

# Step 8: Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, num_workers=4)

print("Data loading complete! Ready for SpinalSAENet training.")

In [ ]:
joblib.dump(scaler, 'scaler.pkl')

# Model training

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize

# Define Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.5, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.reduction = reduction
        self.alpha = alpha  # Class weights

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(weight=self.alpha, reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss

# Define SpinalSAENet (unchanged)
class SpinalSAENet(nn.Module):
    def __init__(self, input_dim, hidden_dims=[128, 64], num_classes=5):
        super(SpinalSAENet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dims[1], 32),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        output = self.classifier(encoded)
        return output

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Compute input_dim and num_classes
input_dim = train_dataset.tensors[0].shape[1]  # 11
num_classes = len(torch.unique(train_dataset.tensors[1]))  # Should be 5
print(f"Input dimension: {input_dim}, Number of classes: {num_classes}")

# Initialize model
model = SpinalSAENet(input_dim, hidden_dims=[128, 64], num_classes=num_classes)
model = model.to(device)

# Class weights (logarithmic inverse frequency with adjustment for Class 3)
y_test = test_dataset.tensors[1].numpy()
class_counts = np.bincount(y_test)
print("Class distribution in test set:", {i: count for i, count in enumerate(class_counts)})
class_weights = np.log(1.0 / class_counts + 1e-6)  # Add small constant to avoid log(0)
class_weights = class_weights / class_weights.sum() * len(class_counts)
# Increase weight for Class 3
class_weights[3] *= 2.0  # Double the weight for Class 3
class_weights = torch.FloatTensor(class_weights).to(device)

# Use Focal Loss
criterion_clf = FocalLoss(alpha=class_weights, gamma=2.5)
optimizer_clf = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_clf, mode='min', factor=0.1, patience=3)
scaler = torch.cuda.amp.GradScaler()

print("Training Classifier...")
best_val_loss = float('inf')
patience_counter = 0
patience = 5  # Early stopping patience

for epoch in range(20):
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
        optimizer_clf.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(data)
            loss = criterion_clf(output, target)
        if torch.isnan(loss):
            print("NaN loss detected in training, skipping batch")
            continue
        scaler.scale(loss).backward()
        scaler.step(optimizer_clf)
        scaler.update()
        train_loss += loss.item()
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_targets = []
    all_scores = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
            with torch.cuda.amp.autocast():
                outputs = model(data)
                loss = criterion_clf(outputs, target)
            val_loss += loss.item()
            # Apply threshold adjustment for Class 3
            probs = torch.softmax(outputs, dim=1)
            class_3_threshold = 0.7  # Increase threshold for Class 3
            probs[:, 3] = torch.where(probs[:, 3] >= class_3_threshold, probs[:, 3], torch.tensor(0.0, device=device))
            _, predicted = torch.max(probs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            all_scores.extend(outputs.cpu().numpy())  # Raw logits for ROC
    
    val_loss /= len(test_loader)
    accuracy = np.mean(np.array(all_preds) == np.array(all_targets))
    f1_weighted = f1_score(all_targets, all_preds, average='weighted')
    f1_macro = f1_score(all_targets, all_preds, average='macro')
    precision, recall, f1_per_class, _ = precision_recall_fscore_support(all_targets, all_preds, average=None, zero_division=0)
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss / len(train_loader):.4f}, Val Loss: {val_loss:.4f}, Test Accuracy: {accuracy*100:.2f}%, F1-Weighted: {f1_weighted:.4f}, F1-Macro: {f1_macro:.4f}")
    print("Per-class F1-scores:", {f"Class {i}": f1 for i, f1 in enumerate(f1_per_class)})
    
    scheduler.step(val_loss)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "spinal_saenet_best.pth")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Validation loss not improving, stopping training.")
            break

# Load best model
model.load_state_dict(torch.load("spinal_saenet_best.pth"))
print("Loaded best model with validation loss:", best_val_loss)

# Generate confusion matrix and ROC curves
model.eval()
all_preds = []
all_targets = []
all_scores = []
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device, non_blocking=True), target.to(device, non_blocking=True)
        outputs = model(data)
        # Apply threshold adjustment for Class 3
        probs = torch.softmax(outputs, dim=1)
        class_3_threshold = 0.7  # Increase threshold for Class 3
        probs[:, 3] = torch.where(probs[:, 3] >= class_3_threshold, probs[:, 3], torch.tensor(0.0, device=device))
        _, predicted = torch.max(probs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(target.cpu().numpy())
        all_scores.extend(outputs.cpu().numpy())  # Raw logits





Using device: cuda
Input dimension: 11, Number of classes: 5
Class distribution in test set: {0: 449939, 1: 22, 2: 525176, 3: 124, 4: 24739}
Training Classifier...
Epoch 1, Train Loss: 0.3459, Val Loss: 0.3583, Test Accuracy: 62.38%, F1-Weighted: 0.6179, F1-Macro: 0.7530
Per-class F1-scores: {'Class 0': 0.6466270810627983, 'Class 1': 1.0, 'Class 2': 0.5754307620062492, 'Class 3': 0.544642857142857, 'Class 4': 0.9984242424242424}
Epoch 2, Train Loss: 0.1748, Val Loss: 0.3239, Test Accuracy: 62.90%, F1-Weighted: 0.6046, F1-Macro: 0.7235
Per-class F1-scores: {'Class 0': 0.6870843489004868, 'Class 1': 1.0, 'Class 2': 0.5153113499498161, 'Class 3': 0.416243654822335, 'Class 4': 0.9988280460699132}
Epoch 3, Train Loss: 0.1569, Val Loss: 0.3195, Test Accuracy: 61.84%, F1-Weighted: 0.5895, F1-Macro: 0.6416
Per-class F1-scores: {'Class 0': 0.6853247559731026, 'Class 1': 1.0, 'Class 2': 0.48824143361943834, 'Class 3': 0.035915304207987137, 'Class 4': 0.9986063140035146}
Epoch 4, Train Loss: 0.14

In [83]:
print(f"Final Accuracy: {accuracy*100:.2f}%")

Final Accuracy: 99.84%


In [86]:
model.eval()

SpinalSAENet(
  (encoder): Sequential(
    (0): Linear(in_features=11, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.6, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.6, inplace=False)
    (3): Linear(in_features=32, out_features=5, bias=True)
  )
)

# confusion  matrix of the classifed samples

In [ ]:
# Confusion Matrix
cm = confusion_matrix(all_targets, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# ROC

In [ ]:
# ROC Curves
y_test_bin = label_binarize(all_targets, classes=range(num_classes))
all_scores = np.array(all_scores)

plt.figure(figsize=(8, 6))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], all_scores[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves (One-vs-Rest)")
plt.legend(loc="lower right")
plt.show()

In [92]:
import pickle

# Save the entire model (structure + weights)
model.cpu()  # Move to CPU before pickling
model.eval()

with open('intrusion_Model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model saved as intrusion_Model.pkl")


Model saved as intrusion_Model.pkl


In [ ]:
with open('intrusion_Model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
loaded_model.eval()
loaded_model.to(device)  # move back to GPU if needed


# cusytom input prediciton

In [91]:
import torch
import torch.nn as nn
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# === Define the model architectur

# === Class label mapping === #
label_mapping = {
    0: "Dos",
    1: "Theft",
    2: "DDos",
    3: "Normal",
    4: "Reconnaissance"
}

# === Required features in order === #
required_features = [
    'pkts', 'bytes', 'sbytes', 'dbytes', 'rate',
    'mean', 'stddev', 'sum', 'proto', 'state',
     'dur'
]

# === Load the trained model === #
try:
    # Use same SpinalSAENet class definition as during training
    model = SpinalSAENet(input_dim=11, hidden_dims=[128, 64], num_classes=5).to(device)
    model.load_state_dict(torch.load("/kaggle/working/spinal_saenet_best.pth", map_location=device))
    model.eval()

    print("Model loaded successfully.")
except Exception as e:
    print("Error loading model:", e)
    raise

# === Load the scaler === #
try:
    scaler = joblib.load('/kaggle/working/scaler.pkl')
    print("Scaler loaded successfully.")
except Exception as e:
    print("Error loading scaler:", e)
    raise

# === Direct custom input (your example) === #
custom_input = {
    'pkts': 100,
    'bytes': 5000,
    'sbytes': 3000,
    'dbytes': 2000,
    'rate': 50.0,
    'mean': 25.0,
    'stddev': 10.0,
    'sum': 10000,
    'proto': 0,  # Encoded value (e.g., 0 for TCP)
    'state': 1,  # Encoded value (e.g., 1 for CON)
    # 'stime': 1620000000,
    'dur': 0.5
}

# === Prediction pipeline === #
try:
    # Convert input to array
    input_array = np.array([[custom_input[feat] for feat in required_features]], dtype=np.float32)

    # Scale the input
    input_scaled = scaler.transform(input_array)

    # Convert to torch tensor
    X_tensor = torch.tensor(input_scaled, dtype=torch.float32).to(device)

    # Predict
    with torch.no_grad():
        outputs = model(X_tensor)
        probs = torch.softmax(outputs, dim=1).cpu().numpy().flatten()
        predicted_class = int(np.argmax(probs))
        confidence = probs[predicted_class]

    print("\n=== Prediction Result ===")
    print(f"Predicted Label: {label_mapping[predicted_class]}")
    print(f"Confidence: {confidence:.4f}")
    print("Class Probabilities:")
    for idx, prob in enumerate(probs):
        print(f"  {label_mapping[idx]}: {prob:.4f}")

except Exception as e:
    print("Prediction failed:", e)


Using device: cuda
Model loaded successfully.
Scaler loaded successfully.

=== Prediction Result ===
Predicted Label: Normal
Confidence: 1.0000
Class Probabilities:
  Dos: 0.0000
  Theft: 0.0000
  DDos: 0.0000
  Normal: 1.0000
  Reconnaissance: 0.0000
